# Job Scheduler

## Overview:

- **Teaching:** 30 min
- **Exercises:** 30 min

**Questions**
- What is a scheduler and why are they used?
- How do I launch a program to run on a compute node?
- How do I check limits on the cluster and my accounts?

**Objectives**
- Run a simple job on the cluster's compute nodes.
- Inspect the status of your job.
- Inspect the output and error files of your job.

# Job Scheduling

An HPC system might have thousands of nodes and thousands of users. How do we decide who gets what and when? How do we ensure that a task is run with the resources it needs? This job is handled by a special piece of software called the scheduler. On an HPC system, the scheduler manages which jobs run where and when.

The following illustration compares these tasks of a job scheduler to a waiter in a restaurant. If you can relate to an instance where you had to wait for a while in a queue to get in to a popular restaurant, then you may now understand why sometimes your jobs do not start instantly as in your laptop.

![Scheduler](../images/03/restaurant_queue_manager.svg)

### Slurm: Simple Linux Utility for Resource Managment

Interacting with the scheduler is done through the terminal using an array of commands. If multiple jobs ran on a single node at the same time users would be competing for the same resources and jobs take longer to run overall. A scheduler manages individual jobs, which are allocated to the resources they need as they become available. This results in a higher overall throughput and more consistent performance.

The scheduler on our cluster container (and Nimbus) is `slurm`: _**S**_imple _**L**_inux _**U**_tility for _**R**_esource _**M**_anagment.

Interacting with the scheduler is done through the terminal using an array of commands. 

## Exercise: Run a bash job

Using the text editor in the terminal create a bash script called `test_job.sh` with the following:

```bash
#!/bin/bash

echo 'This script is running on:'
hostname
sleep 60
```

And run it.

Where did the script run? 


## Exercise: sbatch

If you completed the previous challenge successfully, you probably realise that there is a distinction between running the job through the scheduler and just “running it”. To submit this job to the scheduler, we use a command to submit the job to slurm:

```bash
jupyter-user:$ sbatch test_job.sh
```

Try this now. What happens?


## Solution: sbatch
We get the following error:

```bash
sbatch: error: Batch job submission failed: Invalid qos specification
```



When submitting a job you typically have to give the scheduler, slurm, information in order to help it decide where and when to run the job. 

If there are lots of different compute nodes, for example, we might want to make sure our job goes on a particular CPU. And we will have to tell `slurm` how long our job will run for so it can make decisions about where to put jobs. 

So lets have a look at a more complete job script:

```bash
#!/bin/bash

#SBATCH --account=prj0_phase1
#SBATCH --job-name=myjob
#SBATCH --partition=shortJob
#SBATCH --qos=shortJob
#SBATCH --time=0-00:25:00

echo 'This script is running on:'
hostname
sleep 60
```

Every line that starts with a `#SBATCH` is a directive for `slurm` this is where we tell slurm the resources we want for our job. 

`#SBATCH --account=` tells slurm what account you wish to run against. Slurm will select a default account, but it is better practice to make sure you include this. (In order to run a job on Nimbus you need to have a resource allocation, with funds, and it is this code that you put here)

`#SBATCH --job-name=` gives the job a name, which you can identify in the queue - you can call the job whatever you want that helps you to identify it. 

`#SBATCH --partition=` tells slurm what partition to put the job on. Typically a HPC system will have different partitions which will have different resources or limits on them. For example, there may be a partion with a particular CPU type, or one with limits on the size or duration of job you can run. 

`#SBATCH --qos=` tells slurm what Quality of Service you wish to run with. The QOS is simply a way for admins to apply rules to the resources you can access, and the priority of the job. For Bath's HPC systems the qos will match the partition name - and **remember** if you don't include it you will get an error.

`#SBATCH --time=` tells slurm how long you wish to run the job for, with several acceptable formats:  "minutes", "minutes:seconds", "hours:minutes:seconds", "days-hours", "days-hours:minutes" and "days-hours:minutes:seconds" e.g. `1-01:20:00` will request a runtime of 25 hours and 20 mins. 


## Exercise: Running a batch job

Try editing your runscript to match the example above and submit it. 

What happens?

## Solution: Running a batch job

Again we get an error. 

```sbatch: error: Batch job submission failed: Requested time limit is invalid (missing or exceeds some limit)```

## sinfo

We get an error above because the time limit on the partition has been exceeded. We can use a `slurm` command `sinfo` to get information about what partitions are available:

```bash
jupyter-user:$sinfo
PARTITION      AVAIL  TIMELIMIT  NODES  STATE NODELIST
PARTITION     AVAIL  TIMELIMIT  NODES  STATE NODELIST
bigJob           up 1-00:00:00      4   idle c[1-4]
shortJob         up      15:00      4   idle c[1-4]
notallowedjob    up      15:00      4   idle c[1-4]
```

We can see there are three different partitions. `bigJob`, `shortJob` and `notallowedjob`. All are available (we can see up in `AVAIL` column), next we can see the time limit on jobs allowed to run un that partition. 

Now we can see why our job was rejected - we requested a 25 minute runtime on the `shortjob` partition, when the timelimit is only 15 minutes. 
Try reducing the time requested in the runscript and resubmitting. 

## sacctmgr

Yet another error:

```bash
sbatch: error: Batch job submission failed: Invalid account or account/partition combination specified
```

So we need to check what accounts and QOS we have access to. We can do that with the `sacctmgr` command:

```
sacctmgr show associations user=jupyter-user --parsable2 format=account,user

```

This shows the associations between the QOS and our user account, output only what we are interested in (`format=account,user,QOS`) in a readbale format (`--parsable2`). 



## Exercise: adding the account

Using the sacctmgr command take note of the account you have access to and update your submission script, and resunbmit it using the sbatch command. 

Where was the script executed? Have any new files been created?

## Solution: adding the account

If you have successfully updated your runscript it should now be able to be submitted, and should run on one of the compute nodes (c*). 

A file named `slurm-<jobid>.out` will have been created. This is the stdout from your run. 

## Redirecting output

We have seen that a new file was created in our successful submission `slurm-<jobid>.out` we can tailor where our output goes with a couple of slurm directives:

`#SBATCH --output=` will tell where to put the output

`#SBATCH --error=` will tell slurm where to send any error messages

We can also incorporate the job name with `%x` and the job id with `%j` to help identify our output files and ensure they arent overwritten:





## Exercise: Redirecting output

Include the following in your run script and resubmit.

```bash
#SBATCH --output=%x.%j.o
#SBATCH --error=%x.%j.e
```

What happens to your output now?

## Solution: Redirecting output

Your output should have been sent to a file called `myjob.<jobid>.out` where `<jobid>` is a numeric value identofying your job. 

```bash
#!/bin/bash

#SBATCH --account=prj0_phase1
#SBATCH --job-name=JOB_NAME
#SBATCH --output=%x.%j.o
#SBATCH --error=%x.%j.e
#SBATCH --nodes=4
#SBATCH --ntasks-per-node=1
#SBATCH --partition=bigJob
#SBATCH --qos=bigjob
#SBATCH --time=04:00:00

module load mpi/openmpi-x86_64
echo hello
mpirun -np 4 hello_mpi
sleep 10
```